In [ ]:
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from math import log2
from sklearn.metrics import accuracy_score

In [ ]:
class Parser:

    def __init__(self):
        self.dataMatrix = None
        self.labels = None


    def getLexicalSize(self,file):
        lexiconS = []
        with open(file, "r") as fp:
            for line in fp:
                lexiconS.append(line.split()[0])
        return  len(lexiconS)
    def getLabels(self,data):
        self.labels = []
        with open(data, "r") as fp:
            for line in fp:
                self.labels.append([int(i) for i in line.split()])

    def getData(self,data,lexicon):
        bagof = []
        with open(data,"r") as fp:
            for line in fp:
                vertex = [0 for i in range(self.getLexicalSize(lexicon))]
                array = list(filter(lambda x : x[0] != '<' , line.split()))
                for ele in array:
                    vertex[int(ele)] +=1
                bagof.append(vertex)
        return bagof
    def MMNorm(self,data,lexicon):
        self.dataMatrix = []
        minimum = []
        maximum = []
        # Min Max Classification
        for element in self.getData(data,lexicon):
            minimum = min(element)
            maximum = max(element)
            vect = [(i - minimum) / (maximum - minimum) for i in element]
            self.dataMatrix.append(vect)


In [ ]:
labellex  = Parser()
datalex = Parser()

testlabel = Parser()
testdata = Parser()

testlabel.getLabels("/content/test-label.dat")


testdata.MMNorm("/content/test-data.dat","/content/vocabs.txt")

labellex.getLabels("/content/train-label.dat")


datalex.MMNorm("/content/train-data.dat","/content/vocabs.txt")

Y = np.array(labellex.labels)
X = np.array(datalex.dataMatrix)

Y_test = np.array(testlabel.labels)
X_test = np.array(testdata.dataMatrix)


In [ ]:
def get_model(n_inputs, n_outputs,metrics):
    model = Sequential()
    model.add(Dense(n_outputs, input_dim=n_inputs,kernel_regularizer=l2(0.9), activation='relu'))
    model.add(Dense(n_outputs, input_dim=n_outputs,kernel_regularizer=l2(0.9), activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    
    opt = SGD(learning_rate=0.1,momentum= 0.6)
    model.compile(loss='binary_crossentropy',optimizer=opt,metrics=["binary_accuracy","mse","binary_crossentropy"])
    return model


# evaluate a model using repeated k-fold cross-validation
def evaluate_model(x, y,metrics):
    results = list()
    bmodel = 0.0
    bhist = 0.0
    bacc = 0.0
    n_inputs, n_outputs = x.shape[1], y.shape[1]
    # define evaluation procedure
    cv = KFold(n_splits=5)
    # enumerate folds
    for train_ix, test_ix in cv.split(x):
        # prepare data
        x_train, x_test = x[train_ix], x[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        es = EarlyStopping(monitor='val_loss',mode='min',verbose=0,patience=10,min_delta=0.01 )
        model = get_model(n_inputs, n_outputs,metrics)
        # fit model
        
        
        history = model.fit(x_train, y_train,validation_data=(x_test,y_test),batch_size=66, verbose=1, epochs=500,callbacks=[es])
        # make a prediction on the test set
        
       
        scores = model.evaluate(x_test,y_test,verbose=0)
        print(scores)
        if scores[1] > bacc:
   
          bmodel = model
          bhist = history
          bacc = scores[1]
    
    score = bmodel.evaluate(X_test,Y_test,verbose=0)
    plt.plot(bhist.history['loss'], label='train')
    plt.plot(bhist.history['val_loss'], label='test')
    plt.legend()
    plt.show()
    print(score[1],"\n",score[2],"\n",score[3])
    return results,history


#todo
# use the real test data
# fix gpu problem

# reconsider the acc, mean and std

# evaluate model

#results1=evaluate_model(X, Y,tf.keras.metrics.MeanSquaredError())
results2,history=evaluate_model(X, Y,tf.keras.metrics.BinaryAccuracy())
#results3=evaluate_model(X, Y,tf.keras.metrics.BinaryCrossentropy())

#Stats(results1)

#Stats(results3)
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')


# summarize performance